In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install split-folders

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten, LeakyReLU
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import split_folders
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

from keras.callbacks import ModelCheckpoint

In [ ]:
driveZip = '/content/drive/My Drive/temp/cropped_images.zip'
train_path = '/content/output/train/'
valid_path = '/content/output/val/'


image_types = ["kurti","saree","shirt","tshirt"]
epochs = 10
batch_size = 32
image_size = [224,224]
num_classes = 4

In [ ]:
import zipfile
with zipfile.ZipFile(driveZip,'r') as zip_dir:
    zip_dir.extractall(path='/content/')

In [6]:
split_folders.ratio('cropped_images', output="output", seed=1337, ratio= (.8,.2)) # default values

Copying files: 6383 files [00:02, 2445.76 files/s]


In [ ]:
!rm -r /content/output/train/none
!rm -r /content/output/val/none

In [ ]:
# Image size should be [224,224]
# Iterate through each color folder

def resize_images(image_dir):
  for im_type in image_types:
    # Iterate through each image file in each image_type folder
    #  glob reads in any image with the extension "image_dir/im_type/*"
    for file in glob.glob(os.path.join(image_dir, im_type, "*")):
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((224,224), Image.ANTIALIAS)
        imResize.save(f + '.png', 'PNG', quality=90)

resize_images('/content/output/val')
resize_images('/content/output/train')

In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=image_size + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [26]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(1000)(x)
x = LeakyReLU(alpha=0.05)(x)
prediction = Dense(num_classes, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [29]:
training_set = train_datagen.flow_from_directory('/content/output/train/',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 4162 images belonging to 4 classes.


In [30]:
validation_set = val_datagen.flow_from_directory('/content/output/val/',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 1042 images belonging to 4 classes.


In [ ]:
#earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
'''model.load_weights("my_vgg_run3.h5");
epochs = 5'''

In [32]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=validation_set,
  epochs=epochs,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set),
  callbacks=[mcp_save]
)

Epoch 1/10
131/131 [==============================] - 60s 461ms/step - loss: 1.8775 - accuracy: 0.7729 - val_loss: 0.4302 - val_accuracy: 0.8532
Epoch 2/10
131/131 [==============================] - 58s 444ms/step - loss: 0.2928 - accuracy: 0.8948 - val_loss: 0.4874 - val_accuracy: 0.8647
Epoch 3/10
131/131 [==============================] - 59s 451ms/step - loss: 0.2593 - accuracy: 0.9046 - val_loss: 0.6672 - val_accuracy: 0.8848
Epoch 4/10
131/131 [==============================] - 59s 450ms/step - loss: 0.2234 - accuracy: 0.9169 - val_loss: 0.7910 - val_accuracy: 0.7898
Epoch 5/10
131/131 [==============================] - 59s 451ms/step - loss: 0.2927 - accuracy: 0.9041 - val_loss: 0.4754 - val_accuracy: 0.8896
Epoch 6/10
131/131 [==============================] - 59s 451ms/step - loss: 0.1258 - accuracy: 0.9515 - val_loss: 0.6062 - val_accuracy: 0.8791
Epoch 7/10
131/131 [==============================] - 59s 450ms/step - loss: 0.1263 - accuracy: 0.9510 - val_loss: 0.4873 - val_ac

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [34]:
secondLastModel = Model(inputs=vgg.input, outputs=x)
secondLastModel.summary()
secondLastModel_json =secondLastModel.to_json()
with open("secondLastModel.json", "w") as json_file:
    json_file.write(secondLastModel_json)
secondLastModel.save_weights('secondLastModel.h5')

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [35]:
inputs = Input(shape=(1000,))
outputs =Dense(num_classes, activation='softmax')(inputs)
lastModel = Model(inputs=inputs, outputs=outputs)
lastModel.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 4004      
Total params: 4,004
Trainable params: 4,004
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lastModel.layers[1].set_weights(model.layers[22].get_weights());

In [ ]:
lastModel_json = lastModel.to_json()
with open("lastModel.json", "w") as json_file:
    json_file.write(lastModel_json)
lastModel.save_weights('lastModel.h5')

In [ ]:
!cp model.json '/content/drive/My Drive/temp/VGG'
!cp secondLastModel.json '/content/drive/My Drive/temp/VGG'
!cp lastModel.json '/content/drive/My Drive/temp/VGG'
!cp model.h5 '/content/drive/My Drive/temp/VGG'
!cp secondLastModel.h5 '/content/drive/My Drive/temp/VGG'
!cp lastModel.h5 '/content/drive/My Drive/temp/VGG'

In [135]:
'''from keras.models import model_from_json

json_file = open('lastModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
last_model = model_from_json(loaded_model_json)

last_model.load_weights("lastModel.h5")'''


'from keras.models import model_from_json\n\njson_file = open(\'lastModel.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nlast_model = model_from_json(loaded_model_json)\n\nlast_model.load_weights("lastModel.h5")'